In [47]:
import pandas as pd
import numpy as np
import glob, os
from operator import itemgetter

data_path='/nfs/zorba/DOWNLOADED/OpenfMRI/openfmri/'
# Below will capture all the structural and functional dicom folders (some are duplicates)

#T1_glob_data = '*/*/*.nii.gz' # Grabs all baseline, later can add the repeat session
T1_glob_data = '*/*/anonymized_reorient.nii.gz' # Grabs all baseline, later can add the repeat session

# below for the ds id. MAKE SURE ALL DS FOLDERS ARE 15 CHARACTERS LONG OR WON'T WORK
ds_glob_data = 'ds*/uncompressed'
os.chdir(data_path)

ds_fols=sorted(glob.glob(ds_glob_data))
T1_sub_ids=[]; 
subject_dict = {}
count=0
scan_count=0
# Here cd into each ds fol
for fol in ds_fols:
    os.chdir(data_path)
    os.chdir(fol)
    # Will use below from making subject label
    ds_id=fol[0:2] + fol[5:8]
    
    # Now grab all scans and match with participants.tsv in each folder 
    T1_scans=sorted(glob.glob(T1_glob_data)) # Focus on T1 scans for now
    print "num of structural scans in fol " + fol + " is: " + str(len(T1_scans))
    scan_count += len(T1_scans)
    # Now here must crawl through participants.tsv files to extract age and sex
    demo_file = 'participants.tsv'
    df = pd.read_csv(demo_file,delimiter='\t',dtype=str)
    try:
        df = df.set_index('participant_id')
    except:
        df = df.set_index('id')

    id_list = df.index.tolist()
    
    # Here can try to match multiple ways of listing age and gender, sex
    # and get the appropriate keys depending on the spelling for particular dataset
    age_vars=['Age','age','age at baseline','Age (years)']
    age_key=list(set(df.keys().tolist()).intersection(age_vars))[0]       
    if age_key == []:
        print "Ooops had trouble grabbing age key"
    
    sex_vars=['sex','gender','Gender','Sex']
    sex_key=list(set(df.keys().tolist()).intersection(sex_vars))[0]
    if sex_key == []:
        print "Oops had trouble grabbing sex key"
        
    # Below can add more if want to get other variables like diagnosis
    
    # Now get only unique ids for structural scans. Get both a ds ID and a sub ID
    for s in T1_scans:
        sub_id=s.split('/')[0]
        xnat_sub_id=ds_id + '_' + sub_id
        
        # now verify that the listed subject folder is listed in the participants file
        if sub_id in id_list:
            count += 1
            df_index = id_list.index(sub_id)
            sex = df.iloc[df_index][sex_key]
            age = df.iloc[df_index][age_key]
            
            if sex != 'n/a' and age != 'n/a' and sex != 'D': 
                subject_dict[xnat_sub_id] = {'age':"",'sex':"",'T1_path':""}
                subject_dict[xnat_sub_id]['age'] = age
                
                # Here make sex canonical
                male_vars=['M','Male','male','MALE','m']
                female_vars = ['F','Female','female','FEMALE','f']
                
                if sex in male_vars:
                    new_sex = 'Male'
                elif sex in female_vars:
                    new_sex = 'Female'
                else:
                    print "error in canonizing sex"
                    print sex
                
                subject_dict[xnat_sub_id]['sex'] = new_sex
                subject_dict[xnat_sub_id]['T1_path'] = fol + '/' + s
        else:
            print sub_id + " not participants id list"
          
              
print (count, len(T1_scans))
print len(subject_dict)
print "total number of globbed scans is: " + str(scan_count)
print "total number of scans with sex and age is: " + str(len(subject_dict))


subject_dict['ds236_sub-14']['age']='89' # instead of 89+
subject_dict['ds236_sub-18']['age']='89'

for key, value in subject_dict.items():
    print key  
    print value["age"]
    print value["sex"]
    print value["T1_path"]


num of structural scans in fol ds000001_R2.0.4/uncompressed is: 15
num of structural scans in fol ds000002_R2.0.5/uncompressed is: 15
num of structural scans in fol ds000003_R2.0.2/uncompressed is: 13
num of structural scans in fol ds000005_R2.0.1/uncompressed is: 16
num of structural scans in fol ds000006_R2.0.1/uncompressed is: 14
num of structural scans in fol ds000007_R2.0.1/uncompressed is: 20
num of structural scans in fol ds000008_R2.0.0/uncompressed is: 14
num of structural scans in fol ds000009_R2.0.2/uncompressed is: 24
num of structural scans in fol ds000011_R2.0.1/uncompressed is: 14
num of structural scans in fol ds000030_R1.0.4/uncompressed is: 263
num of structural scans in fol ds000053_R1.0.0/uncompressed is: 58
num of structural scans in fol ds000101_R2.0.0/uncompressed is: 21
num of structural scans in fol ds000102_R2.0.0/uncompressed is: 26
num of structural scans in fol ds000108_R2.0.0/uncompressed is: 34
num of structural scans in fol ds000109_R2.0.2/uncompressed i

In [59]:
single_subject_dict={}
single_subject_dict = { your_key: subject_dict[your_key] for your_key in ['ds236_sub-18'] }
#single_subject_dict['ds236_sub-18']=subject_dict['ds236_sub-18']
print single_subject_dict
print single_subject_dict.keys()
print single_subject_dict.values()

{'ds236_sub-18': {'age': '89', 'ds236_sub-18': {...}, 'T1_path': 'ds000236_R1.0.0/uncompressed/sub-18/anat/anonymized_reorient.nii.gz', 'sex': 'Female'}}
['ds236_sub-18']
[{'age': '89', 'ds236_sub-18': {...}, 'T1_path': 'ds000236_R1.0.0/uncompressed/sub-18/anat/anonymized_reorient.nii.gz', 'sex': 'Female'}]


In [23]:
# Hopefully just need to run this cell once and then at top change the glob to grab the anonymized_reorient.nii.gz files
import os
os.chdir(data_path)
count=0
for key, value in subject_dict.items():
    count += 1
    print key  
    print value["age"]
    print value["sex"]
    # Here go ahead and run fslreorient2std on all T1 scans and save in same folder,
    # and can run slicesdir on them
    base_fol=os.path.dirname(value["T1_path"])
    img_reorient=os.path.join(base_fol,'anonymized_reorient.nii.gz')
    cmd_reorient='fslreorient2std ' + value["T1_path"] + ' ' + img_reorient
    print "reorienting " + value["T1_path"]
    os.system(cmd_reorient)
    print str(count)


ds217_sub-Exp2s08
31
M
reorienting ds000217_R1.0.1/uncompressed/sub-Exp2s08/anat/sub-Exp2s08_T1w.nii.gz
1
ds217_sub-Exp2s09
23
F
reorienting ds000217_R1.0.1/uncompressed/sub-Exp2s09/anat/sub-Exp2s09_T1w.nii.gz
2
ds217_sub-Exp2s06
20
F
reorienting ds000217_R1.0.1/uncompressed/sub-Exp2s06/anat/sub-Exp2s06_T1w.nii.gz
3
ds217_sub-Exp2s07
20
F
reorienting ds000217_R1.0.1/uncompressed/sub-Exp2s07/anat/sub-Exp2s07_T1w.nii.gz
4
ds217_sub-Exp2s04
20
M
reorienting ds000217_R1.0.1/uncompressed/sub-Exp2s04/anat/sub-Exp2s04_T1w.nii.gz
5
ds217_sub-Exp2s05
21
M
reorienting ds000217_R1.0.1/uncompressed/sub-Exp2s05/anat/sub-Exp2s05_T1w.nii.gz
6
ds217_sub-Exp2s02
20
F
reorienting ds000217_R1.0.1/uncompressed/sub-Exp2s02/anat/sub-Exp2s02_T1w.nii.gz
7
ds217_sub-Exp2s03
23
F
reorienting ds000217_R1.0.1/uncompressed/sub-Exp2s03/anat/sub-Exp2s03_T1w.nii.gz
8
ds217_sub-Exp2s01
25
M
reorienting ds000217_R1.0.1/uncompressed/sub-Exp2s01/anat/sub-Exp2s01_T1w.nii.gz
9
ds115_sub-06
22.8172
FEMALE
reorienting ds0001

In [60]:
import pyxnat, os, subprocess
delete_previous_scans = 0
insert_scans = 1
os.chdir(data_path)

# Set project name
project_name="OPENFMRI"

# connect to XNAT instance
xnat=pyxnat.Interface(config='/home/spantazatos/xnat_sp.cfg')

# Create the project in XNAT
# Create project object
project=xnat.select.project(project_name)

# First check if it already exists
if not project.exists():
    project.insert()
    
for key, value in single_subject_dict.items():
    print key
    print value["T1_path"]
    
    # Now insert subject-level attributes (i.e. SEX) and experiment-level attributes (i.e. AGE)
    #user_age = round(float(value['age']),1)
    try:
        user_age = int(round(float(value['age']),0))
    except:
        continue
    
    user_sex = value['sex']
    
    # Insert subject if not already there
    print ("Now insert subject with key: " + key)
    subject = project.subject(key)
    if not subject.exists():
        subject.insert()
    
    # Since this is baseline just do MR1 for now
    session_label = "%s_%s_MR1" % (project_name,key)
    print session_label
    
    experiment=subject.experiment(session_label) #.create(experiments='xnat:mrSessionData')
   
    # Create experiment object, insert if it doesn't already exst
    if not experiment.exists():
        experiment.create() # By default it will make an xnat:mrSessionData experiment which is what we need
    

        
#     sex = 'Male'
#     weight = str(100)
#     height = str(100)
    
    subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/gender', user_sex)
    subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/age', str(user_age))
    #subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/height', height)
    
    #model ='SIEMENS MAGNETOM TrioTim syngo MR B17'
    #site ='Hammersmith Hospital'
    #fs = value['FS']
    
    #experiment.attrs.set('xnat:imageSessionData/scanner',model)
    #experiment.attrs.set('xnat:imageSessionData/fieldStrength',fs)

    # Below for inserting scans
    if insert_scans==1:
#         scan_files=[]
#         scan_files.append(value['T1_path'])
#         if value['faces_path'] != []:
#             scan_files.append(value['func_path'])
        
    
        # Delete old scans if desired
        if delete_previous_scans==1:
            allScans=experiment.scans()
            for scan in allScans:
                print "deleting scans"
                scan.delete()

        # Here set certain xnat parameters
        scan_type='T1'
        scan_name='T1_01'
        slice_size='6 1200'
            
        img_reorient='anonymized_reorient.nii.gz'
        # rm img_reorient
        cmd_rm='rm anonymized_reorient.nii.gz tmp.gif tmp.png tmp_thumb.gif'
        #cmd_rm='rm anonymized_reorient.nii.gz tmp.gif tmp.png tmp_thumb.gif temp.nii.gz'
        os.system(cmd_rm)
        #cmd_reorient='fslreorient2std ' + value['T1_path'] + ' ' + img_reorient
        #cmd_rename='cp ' + value[func_key] + ' temp.nii.gz' # Don't reorient since already including dcm2nii orientation
        cmd_rename='cp ' + value['T1_path'] + ' ' + img_reorient
        print cmd_rename
        os.system(cmd_rename)

        #cmd_reorient1 = 'flirt -in temp.nii.gz -ref $T1_hires -dof 6 -omat out.mat'
        #print "now flirting"
        #os.system(cmd_reorient1)
        #cmd_reorient2 = 'flirt -in temp.nii.gz -out anonymized_reorient -ref temp.nii.gz -applyxfm -init out.mat -noresample'
        #os.system(cmd_reorient2)

        #sform = subprocess.check_output(['fslorient', '-getsform',img_reorient]).split(' ')
        #0 0 1.1 -87.6105 -1.09375 0 0 166.432 0 1.09375 0 -128.059 0 0 0 1
        #new_sform_list=[sform[4],'0','0','90','0',sform[9],'0','-125','0','0',sform[2],'-67.75','0','0','0','1'] 

        #new_sform=' '.join(new_sform_list).rstrip()
        #cmd_reorient = 'fslorient -setsform ' + new_sform + ' ' + img_reorient + ' && fslorient -setqform ' + new_sform + ' ' + img_reorient
        #print cmd_reorient
        #os.system(cmd_reorient)
        #print subprocess.check_output(['fslorient', '-getsform',img_reorient])

        # Now insert the reoriented image
        # experiment.scan('1').resource('NIFTI').file(os.path.basename(scans_filtered[sub])).insert(
                
        experiment.scan(scan_name).resource('NIFTI').file(os.path.basename(img_reorient)).insert(
        img_reorient,
        content=scan_type,
        overwrite=True,
        type=scan_type,  # May need to change this to SPGR, MPRAGE, FLAIR etc?
        format='NIFTI')

        # Create ORIGINAL and SNAPSHOT to view the images in the web browser
        img_png='tmp.png'
        img_gif='tmp.gif'
        img_thumb='tmp_thumb.gif'
        cmd_create_slices='slicer ' + img_reorient + ' -S ' + slice_size + ' ' + img_png
        cmd_convert_gif='convert ' + img_png + ' ' + img_gif
        cmd_convert_thumb='convert ' + img_png + ' -resize 240x240 ' + img_thumb

        print cmd_create_slices
        print cmd_convert_gif
        print cmd_convert_thumb

        os.system(cmd_create_slices)
        os.system(cmd_convert_gif)
        os.system(cmd_convert_thumb)

        experiment.scan(scan_name).resource('SNAPSHOTS').file(os.path.basename(img_gif)).insert(
        img_gif,
        content='ORIGINAL',
        overwrite=True,
        format='GIF')

        experiment.scan(scan_name).resource('SNAPSHOTS').file(os.path.basename(img_thumb)).insert(
        img_thumb,
        content='THUMBNAIL',
        overwrite=True,
        format='GIF')
                                                    

#xnat.select.project('demoProj_1').accessibility()
#xnat.select.project('demoProj_1').set_accessibility(accessibility='private')

#db.select.project('IXI_TEST').create()
print "ok"
    

ds236_sub-18
ds000236_R1.0.0/uncompressed/sub-18/anat/anonymized_reorient.nii.gz
Now insert subject with key: ds236_sub-18
OPENFMRI_ds236_sub-18_MR1
cp ds000236_R1.0.0/uncompressed/sub-18/anat/anonymized_reorient.nii.gz anonymized_reorient.nii.gz
slicer anonymized_reorient.nii.gz -S 6 1200 tmp.png
convert tmp.png tmp.gif
convert tmp.png -resize 240x240 tmp_thumb.gif
ok


In [63]:
import subprocess
os.chdir('/nfs/zorba/DOWNLOADED/')
img_name = 'anonymized_reorient2'
sform = subprocess.check_output(['fslorient', '-getsform',img_name]).split(' ')
print "old sform is: " + ' '.join(sform)

sform[3] = '90'
sform[7] = '-125'
sform[11] = '-67.75'

new_sform=' '.join(sform).rstrip()
print "new sform is: " + new_sform
cmd_reorient = 'fslorient -setsform ' + new_sform + ' ' + img_name + ' && fslorient -setqform ' + new_sform + ' ' + img_name
#cmd_reorient = 'fslorient -setsform ' + new_sform + ' ' + img_name
print cmd_reorient
os.system(cmd_reorient)
print subprocess.check_output(['fslorient', '-getsform',img_name])


old sform is: -1 -0 -0 -0 -0 1 -0 -0 0 0 1.2 -0 0 0 0 1 

new sform is: -1 -0 -0 90 -0 1 -0 -125 0 0 1.2 -67.75 0 0 0 1
fslorient -setsform -1 -0 -0 90 -0 1 -0 -125 0 0 1.2 -67.75 0 0 0 1 anonymized_reorient2 && fslorient -setqform -1 -0 -0 90 -0 1 -0 -125 0 0 1.2 -67.75 0 0 0 1 anonymized_reorient2
-1 -0 -0 90 -0 1 -0 -125 0 0 1.2 -67.75 0 0 0 1 

